In [27]:
128,862,1

(128, 862, 1)

In [28]:
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense,Flatten,Reshape,Activation,BatchNormalization,Dense,Dropout,Flatten
from tensorflow.keras.layers import LeakyReLU,Conv2D,Conv2DTranspose
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [29]:
img_rows = 862     # 431  #
img_cols = 128     # 64   # 32   # 16
img_channels = 1
img_shape = (img_rows, img_cols, img_channels)

z_dim = 1000

In [30]:
def build_generator(img_shape,s_dim):
    model = Sequential()
    model.add(Dense(16*431,input_dim=z_dim))
    model.add(Reshape((16,431,1)))
    model.add(Conv2DTranspose(128,kernel_size=3,strides=2,padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2DTranspose(64,kernel_size=3,strides=(2,1),padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2DTranspose(1,kernel_size=3,strides=(2,1),padding = 'same'))
    model.add(Activation('tanh'))
    return model

In [31]:
model = build_generator(img_shape,1000)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6896)              6902896   
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 431, 1)        0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 32, 862, 128)      1280      
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 862, 128)      512       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32, 862, 128)      0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 64, 862, 64)       73792     
_________________________________________________________________
batch_normalization_5 (Batch (None, 64, 862, 64)      

In [32]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32,kernel_size=3,strides=2,input_shape = img_shape,padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2D(64,kernel_size=3,strides=2,padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Conv2D(128,kernel_size=3,strides=2,padding='same'))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    return model



In [33]:
model = build_discriminator(img_shape)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 431, 64, 32)       320       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 431, 64, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 216, 32, 64)       18496     
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 216, 32, 64)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 108, 16, 128)      73856     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 108, 16, 128)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 221184)           

In [34]:
def build_gan(generator,discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [35]:
discriminator = build_discriminator(img_shape)
discriminator.compile(loss = 'binary_crossentropy',optimizer = Adam(),metrics=['accuracy'])
generator = build_generator(img_shape,z_dim)
discriminator.trainable=False
gan = build_gan(generator,discriminator)
gan.compile(loss = '',optimizer = Adam())

In [36]:
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations,batch_size,sample_interval):
    (x_train,_),(_,_) = mnist.load_data()
    x_train = x_train/127.5-1.0
    x_train = np.expand_dims(x_train,axis=3)
    
    real = np.ones((batch_size,1))
    fake = np.zeros((batch_size,1))
    
    for iteration in range(iterations):
        idx = np.random.randint(0,x_train.shape[0],batch_size)
        imgs = x_train[idx]
        
        z = np.random.normal(0,1,(batch_size,100))
        gen_imgs = generator.predict(z)
        
        d_loss_real = discriminator.train_on_batch(imgs,real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs,fake)
        d_loss,accuracy = 0.5*np.add(d_loss_real,d_loss_fake)
        
        z = np.random.normal(0,1,(batch_size,100))
        # gen_imgs = generator.predict(z)
        g_loss = gan.train_on_batch(z,real)
        
        if (iteration+1)%sample_interval==0:
            losses.append((d_loss,g_loss))
            accuracies.append(100*accuracy)
            iteration_checkpoints.append(iteration+1)
            
            print("%d [D 손실: %f, 정확도 : %.2f%%]  [G 손실 : %f]"%(iteration+1,d_loss,100*accuracy,g_loss))
            
            sample_images(generator)

In [37]:
def sample_images(generator,image_grid_rows = 4,image_grid_columns=4):
    z = np.random.normal(0,1,(image_grid_rows*image_grid_columns,z_dim))
    gen_imgs = generator.predict(z)
    gen_imgs = 0.5*gen_imgs+0.5
    fig,axs = plt.subplots(image_grid_rows,image_grid_columns,figsize=(4,4),sharey=True,sharex=True)
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
            axs[i,j].axis('off')
            cnt+=1

In [38]:
iterations = 20000
batch_size=128
sample_interval = 1000

train(iterations,batch_size,sample_interval)

ValueError: in user code:

    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
        output